In [1]:
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os

plt.style.use("ggplot")
%matplotlib inline
%load_ext jupyter_black

In [2]:
import datetime


def format_date(date="2021-W12"):
    year = datetime.datetime.strptime(date.split("-")[0], "%Y")
    weeks = datetime.timedelta(weeks=int(date.split("-W")[1]))
    date = year + weeks
    return date

In [3]:
df = pd.read_csv("https://opendata.ecdc.europa.eu/covid19/vaccine_tracker/csv/data.csv")
df["date"] = df.YearWeekISO.apply(format_date)
df = df[df.ReportingCountry == df.Region]
df = df.set_index(["date", "ReportingCountry", "TargetGroup", "Vaccine"])
df = df[
    [
        "Denominator",
        "NumberDosesReceived",
        "NumberDosesExported",
        "FirstDose",
        "FirstDoseRefused",
        "SecondDose",
        "DoseAdditional1",
        "DoseAdditional2",
        "DoseAdditional3",
        "DoseAdditional4",
        "DoseAdditional5",
        "UnknownDose",
        "Population",
    ]
]
df = df.convert_dtypes(int)
df.to_pickle("data/df_ecdc.pkl")

In [4]:
df = pd.read_pickle("data/df_ecdc.pkl")
df = df.replace(pd.NA, 0)
df["NumberDosesInjected"] = df[
    [
        "FirstDose",
        "SecondDose",
        "DoseAdditional1",
        "DoseAdditional2",
        "DoseAdditional3",
        "DoseAdditional4",
        "DoseAdditional5",
        "UnknownDose",
    ]
].sum(axis=1)
ds = df.to_xarray()
ds.to_netcdf("data/ds_ecdc.nc")
ds

<xarray.Dataset>
Dimensions:              (date: 147, ReportingCountry: 30, TargetGroup: 17,
                          Vaccine: 19)
Coordinates:
  * date                 (date) datetime64[ns] 2020-12-16 ... 2023-10-01
  * ReportingCountry     (ReportingCountry) object 'AT' 'BE' 'BG' ... 'SI' 'SK'
  * TargetGroup          (TargetGroup) object '1_Age60+' '1_Age<60' ... 'LTCF'
  * Vaccine              (Vaccine) object 'AZ' 'BECNBG' 'BHACOV' ... 'UNK' 'VLA'
Data variables: (12/14)
    Denominator          (date, ReportingCountry, TargetGroup, Vaccine) object ...
    NumberDosesReceived  (date, ReportingCountry, TargetGroup, Vaccine) object ...
    NumberDosesExported  (date, ReportingCountry, TargetGroup, Vaccine) object ...
    FirstDose            (date, ReportingCountry, TargetGroup, Vaccine) object ...
    FirstDoseRefused     (date, ReportingCountry, TargetGroup, Vaccine) object ...
    SecondDose           (date, ReportingCountry, TargetGroup, Vaccine) object ...
    ...                   ...
    DoseAdditional3      (date, ReportingCountry, TargetGroup, Vaccine) object ...
    DoseAdditional4      (date, ReportingCountry, TargetGroup, Vaccine) object ...
    DoseAdditional5      (date, ReportingCountry, TargetGroup, Vaccine) object ...
    UnknownDose          (date, ReportingCountry, TargetGroup, Vaccine) object ...
    Population           (date, ReportingCountry, TargetGroup, Vaccine) object ...
    NumberDosesInjected  (date, ReportingCountry, TargetGroup, Vaccine) float64 ...